<a href="https://colab.research.google.com/github/rizavelioglu/hateful_memes-hate_detectron/blob/main/notebooks/%5BGitHub%5Dbenchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install MMF**

In [ ]:
!git clone https://github.com/facebookresearch/mmf.git

In [ ]:
import os
os.chdir("mmf")
!pip install --editable .

---
## **Download the dataset (Phase2) & convert it into *MMF* format**

In [ ]:
#@markdown ---
#@title <h1><b><font color='red'> --Action required!-- </b></font></h1> { run: "auto" }
#@markdown First, please specify the download link and the `.zip` password which both can be taken from [DrivenData](https://www.drivendata.org/competitions/70/hateful-memes-phase-2/data/)


YOUR_LINK_TO_DOWNLOAD_PHASE2_DATA = '' #@param {type:"string"}
PASSWORD_OF_ZIP = '' #@param {type:"string"}

In [ ]:
!wget -O XjiOc5ycDBRRNwbhRlgH.zip --no-check-certificate --no-proxy "$YOUR_LINK_TO_DOWNLOAD_PHASE2_DATA"

In [ ]:
!mmf_convert_hm --zip_file="XjiOc5ycDBRRNwbhRlgH.zip" --password=$PASSWORD_OF_ZIP --bypass_checksum 1

---
## **Finetuning from a pretrained model & Generating Submission for the Challenge**
https://mmf.sh/docs/tutorials/checkpointing/


> **Example**
https://github.com/apsdehal/hm_example_mmf


After we trained the model and evaluated on the validation set, we will generate the predictions on the test set. The prediction file should contain the following three columns:

- Meme identification number, id
- Probability that the meme is hateful, proba
- Binary label that the meme is hateful (1) or non-hateful (0), label

> With MMF you can directly generate the predictions in the required submission format with the following command: 

**Note**: This command will output where the generated predictions csv file is stored.

In [ ]:
# Free up the disk by removing .zip, .tar files
!rm -rf /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/features/features.tar.gz
!rm -rf /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/XjiOc5ycDBRRNwbhRlgH.zip
!rm -rf /content/mmf/XjiOc5ycDBRRNwbhRlgH.zip
!rm -rf /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/extras.tar.gz

### <font color='magenta'> <b> VisualBERT </b> </font>

In [ ]:
import os
os.chdir("/content")

!mmf_run config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
        model="visual_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \
        training.tensorboard=True \
        env.tensorboard_logdir="logs/fit/" \
        training.checkpoint_interval=100 \
        training.evaluation_interval=100 \
        checkpoint.max_to_keep=1 \
        training.max_updates=3000 \
        training.log_interval=100 \
        checkpoint.resume_zoo=visual_bert.pretrained.coco.fifty_pc \
        training.lr_ratio=0.3 \
        dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train.jsonl" \
        dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
        dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl" \

##### **Visualize losses/accuracy via Tensorboard**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

##### **Running validation using the trained model**

In [ ]:
!mmf_run config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
    model="visual_bert" \
    dataset=hateful_memes \
    run_type=val \
    checkpoint.resume_file="/content/save/best.ckpt" \
    dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl" \
    dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
    dataset_config.hateful_memes.features.train[0]="/content/features" \
    dataset_config.hateful_memes.features.val[0]="/content/features" \
    dataset_config.hateful_memes.features.test[0]="/content/features" \

##### **Generate predictions for the Challenge**

In [ ]:
!mmf_predict config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
    model="visual_bert" \
    dataset=hateful_memes \
    run_type=test \
    checkpoint.resume_file="/content/mmf/save/best.ckpt"

---
### <font color='magenta'> <b> ViLBERT </b> </font>

In [ ]:
"""
https://github.com/facebookresearch/mmf/tree/master/projects/pretrain_vl_right
Pretrained Model: Pretrained Key
--------------------------------
Masked COCO 100% : vilbert.pretrained.coco
Masked VQA2 100% : vilbert.pretrained.vqa2
Masked CC 100%   : vilbert.pretrained.cc.full
"""

!export OC_DISABLE_DOT_ACCESS_WARNING=1


!mmf_run config="projects/vilbert/configs/hateful_memes/from_cc.yaml" \
        model="vilbert" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \
        training.tensorboard=True \
        env.tensorboard_logdir="logs/fit/" \
        training.checkpoint_interval=500 \
        training.evaluation_interval=200 \
        checkpoint.max_to_keep=1 \
        training.max_updates=10000 \
        training.log_interval=50 \
        checkpoint.resume_zoo=vilbert.pretrained.cc.full \
        training.lr_ratio=0.3 \

##### **Visualize losses/accuracy via Tensorboard**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

---
### <font color='magenta'> <b> MMBT </b> </font>

In [ ]:
"""
  To train MMBT model with Faster RCNN region features on the Hateful Memes dataset, run the following command
"""
# To train MMBT model with grid features on the Hateful Memes dataset, run the following command
#   mmf_run config=projects/mmbt/configs/hateful_memes/defaults.yaml run_type=train_val dataset=hateful_memes model=mmbt
!mmf_run config="projects/mmbt/configs/hateful_memes/with_features.yaml" \
        model="mmbt" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \
        training.tensorboard=True \
        env.tensorboard_logdir="logs/fit/" \
        training.checkpoint_interval=2000 \
        training.evaluation_interval=2000 \

##### **Visualize losses/accuracy via Tensorboard**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

##### **Running validation using the trained model**

In [ ]:
!mmf_run config="projects/mmbt/configs/hateful_memes/with_features.yaml" \
    model="mmbt" \
    dataset=hateful_memes \
    run_type=val \
    checkpoint.resume_file="/content/mmf/save/best.ckpt"

##### **Generate predictions for the Challenge**

In [ ]:
!mmf_predict config="projects/mmbt/configs/hateful_memes/with_features.yaml" \
    model="mmbt" \
    dataset=hateful_memes \
    run_type=test \
    checkpoint.resume_file="/content/mmf/save/best.ckpt"

---
### <font color='magenta'> <b> ConcatBERT </b> </font>

In [ ]:
import os
os.chdir("/content")

!mmf_run config="projects/others/concat_bert/hateful_memes/defaults.yaml" \
        model="concat_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \
        training.tensorboard=True \
        env.tensorboard_logdir="logs/fit/" \
        training.checkpoint_interval=100 \
        training.evaluation_interval=100 \
        checkpoint.max_to_keep=1 \
        training.max_updates=3000 \
        training.log_interval=100 \
        checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
        training.lr_ratio=0.3 \
        dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train.jsonl" \
        dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
        dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl" \

---
### <font color='magenta'> <b> ConcatBOW </b> </font>

In [ ]:
!mmf_run config="projects/others/concat_bow/hateful_memes/defaults.yaml" \
        model="concat_bow" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \

---
### <font color='magenta'> <b> Late Fusion </b> </font>

In [ ]:
!mmf_run config="projects/others/late_fusion/hateful_memes/defaults.yaml" \
        model="late_fusion" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \

---
### <font color='magenta'> <b> CNN-LSTM </b> </font>

In [ ]:
!mmf_run config="projects/others/cnn_lstm/hateful_memes/defaults.yaml" \
        model="cnn_lstm" \
        dataset=hateful_memes \
        run_type=train_val \
        training.batch_size=32 \